In [91]:
import pandas as pd
# pd.set_option('display.max_colwidth', None)
from azure.identity import AzureCliCredential, DefaultAzureCredential
from openai import AzureOpenAI
import os
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import json
#  system_prompt
"""

# Step 2: 导入所需模块
from azure.identity import AzureCliCredential, DefaultAzureCredential
from openai import AzureOpenAI

# Step 3: 获取 Azure CLI 凭据
credential = AzureCliCredential()

"""
# export PATH=$PATH:/home/jason00
def generate_prompt():
    dau_analysis_sql_query=""" SELECT 
    metrics_date, 
    application_id, 
    SUM( CAST( metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION ) ) AS dau 
    FROM 
    your_schema.your_table 
    WHERE metrics_date BETWEEN '2024-06-09' AND '2024-06-09' AND period = 1    
    AND os_version = '#Overall#'     
    AND device_model = '#Overall#'    
    AND client_version = '#Overall#' 
    AND client_build_type = '#Overall#'    
    AND install_channel_l1 = '#Overall#'    
    AND install_channel_l2 = '#Overall#'    
    AND install_channel_l3 = '#Overall#'    
    AND install_channel_l4 = '#Overall#' 
    AND first_launch_source = '#Overall#'    
    AND launch_source = '#Overall#' 
    AND market = '#Overall#'     
    AND application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B') 
    GROUP BY metrics_date, application_id
    LIMIT 10;"""

    wau_analysis_sql_query=""" SELECT 
    metrics_date, 
    application_id, 
    SUM( CAST( metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION ) ) AS dau 
    FROM 
    your_schema.your_table 
    WHERE metrics_date BETWEEN '2024-06-03' AND '2024-06-09' AND period = 7    
    AND os_version = '#Overall#'     
    AND device_model = '#Overall#'    
    AND client_version = '#Overall#' 
    AND client_build_type = '#Overall#'    
    AND install_channel_l1 = '#Overall#'    
    AND install_channel_l2 = '#Overall#'    
    AND install_channel_l3 = '#Overall#'    
    AND install_channel_l4 = '#Overall#'
    AND first_launch_source = '#Overall#'    
    AND launch_source = '#Overall#' 
    AND market = '#Overall#'     
    AND application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B') 
    GROUP BY metrics_date, application_id
    LIMIT 10;"""
    market_top_10_dau_query="""
    SELECT market,
    SUM( CAST( metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION ) ) AS dau 
    FROM 
    your_schema.your_table 
    WHERE metrics_date BETWEEN  '2024-06-09' AND '2024-06-09'
    AND application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B')
    GROUP BY market
    ORDER BY dau DESC
    LIMIT 10;"""
    market_count_dau_query="""
    SELECT count( distinct( market ) )
    FROM 
    your_schema.your_table 
    WHERE metrics_date BETWEEN  '2024-06-09' AND '2024-06-09'
    AND application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B')
    LIMIT 10;
    """
    json_fewshot=[
      {
        "Analysis": {
            "input":"Bing APP一天有多少用户？",
            "description":"获取在2024年6月9日这一天，指定的两个应用程序（Bing-Android和Bing-IOS）的日活跃用户数（DAU）",
            "SQL": dau_analysis_sql_query,
            "date": ["2024-06-09","2024-06-09"],
            "result_count": 2,
        },
      },
      {
        "Analysis": {
            "input":"Bing APP一周有多少用户？",
            "description":"获取在2024年6月3日至6月9日这一周，指定的两个应用程序（Bing-Android和Bing-IOS）的日活跃用户数（DAU）",
            "SQL": wau_analysis_sql_query,
            "date": ["2024-06-03","2024-06-09"],
            "result_count": 2,
        },
      },
    ]


    prompt_dau = """
    Context: 
    Now we have the user data for some apps, these app are search apps, it intagrated the copilot for AI.
    User can also search for information, this app get revenue by ads, and it also have many miniapps for different feature.
    We did some AB test based on user interaction data from an app, 
    which includes various metrics such as user behavior, codex (AI) behavior, and search behavior. 

    The data for each user is structured as a sentence.

    The goal is to generate SQL and more information.
    The query should focus on：
    market:
    Format is "en-us",first two is language, last two is country(xl means other counrty that we can not get info.)
    Eg."en-us" means American people speak English.

    For `application_id`：
    - the user will describe an app name, and you need to use the corresponding ID when you search. The mappings are as follows:
       - 'Bing-Android' -> '2130688B018F4B44BBED68E7A42BBA1E'
       - 'Bing-IOS' -> 'AE427635ADC245AE973038BCB3D7C21B'
       - 'Start-Android' -> '4DC5714ABCAD449BA13A9B701A3CF296'
       - 'Start-IOS' -> '4A5B528B59954AAE8725B509A41FBF1A'
       - 'Copilot-Android' -> 'F185A93DE6764B098D55089F610A3FB8'
       - 'Copilot-IOS' -> 'FC320C411FC12CD4DFBE9A00F3161364'

       If the user does not specify the OS name, you need to get both IDs for the app. For example, if the user inputs "Bing", you should use ['2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B'].


    period: 1 for DAU, 7 for WOW, and 30 for MOM

    metrics_date： if not set in the content, it is today-1, format is "2024-06-07".
    For example, if I don't add anything in the input, you need to get today's date, if today is "2024-06-20",you can use metrics_date = "2024-06-19"

    Data Definition:
    If you need to get all data in this type, use "#Overall#". eg. market="#overall".
    Otherwise, if you need to specify type, use your type. eg. market="en-us".Not only in "market", use it in all type.



    Here is the schema of table(sapphire.sapphire_engagement_metrics_master):                                                                                                                                                                                                                                                                
    umid                 | 1736681
    metrics_date         | 2022-09-23
    period               | 1
    application_id       | 4DC5714ABCAD449BA13A9B701A3CF296
    market               | en-in
    os_version           | #Overall#
    device_model         | #Overall#
    client_version       | 23.3.400914606
    client_build_type    | #Overall#
    install_channel_l1   | #Overall#
    install_channel_l2   | #Overall#
    install_channel_l3   | #Overall#
    install_channel_l4   | #Overall#
    first_launch_source  | #Overall#
    launch_source        | #Overall#
    metrics              | {'ActiveUser': {'uu': {'uu': 467.0}, 'msa': {'msa_uu': 0.0}, 'msb': {'msb_uu': 0.0}, 'session': {'session_cnt': 894.0}, 'dwelltime': {'dwelltime': 107686.0, 'dwelltime_uu': 382.0}}}


    Output Format: 
    Each set of data returns one json, followed by an example based on the provided data.

    result_count: How many result will get after query, for example, if group by 2 date and 3 application, it will get 2*3=6 columns.
    if you guess the column will more than 100, eg. group by market, please use 100.

    Specifications:
    - Analyze the input and describe their demand.
    - Use these column to identify and describe user demand.
    - Provide a detailed and correct pgsql query for each input.
    - Limit the output length to avoid excessive verbosity.
    - `metrics_date`: If not set in the content, it should be yesterday's date in the format "YYYY-MM-DD". For example, if today's date is "2024-06-20", then `metrics_date` should be "2024-06-19".
    - Always use the exact date in the SQL query, do not use relative date expressions like `current_date - INTERVAL '1 day'`.
    - Refer the schema of table. Include the default conditions in every query to ensure accurate calculation by considering all categories
    - To calculate user count, always use `SUM( CAST( metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION ) ) AS dau` instead of `COUNT(DISTINCT umid)`.The `umid` field does not represent user count. 
    - Do not use `#Overall#` for `application_id`. It should always be a specific ID or a list of IDs.
    - Use table sapphire.sapphire_mini_apps_dau_master for miniapp dau

    Task Examples: 


    """
    # json_fewshot_str=json.dumps(json_fewshot,ensure_ascii=False)

    # json_fewshot_str=json.dumps(json_fewshot,ensure_ascii=False)
    for idx,shot in enumerate(json_fewshot):
        prompt_dau+="Input"+str(idx)+"\n"+shot["Analysis"]["input"]+":\n"
        prompt_dau+="Output"+str(idx)+"\n"+json.dumps(shot,ensure_ascii=False)+":\n"
        
    return prompt_dau


# DefaultAzureCredential()
class Agent():
    def __init__(self):
        self.deployment_name="gpt4"
        self.credential = AzureCliCredential()
        self.token_provider = get_bearer_token_provider(
            self.credential, "https://cognitiveservices.azure.com/.default"
        )
        # replace hardcode api key with credential
        self.client4 = AzureOpenAI(
        #     api_version="2024-02-15-preview",
            api_version="2023-12-01-preview",
            azure_endpoint="https://chatapi-openai.openai.azure.com/",
            azure_ad_token_provider=self.token_provider
        )

    def chat(self,query:str,prompt):
        self.messages = self.get_prompt_for_comments([query], prompt)
        # print(str(self.messages))
        # insights = self.get_gpt_response(self.client4, "gpt4", self.messages)
        gpt_response = self.client4.chat.completions.create(
          model="gpt4", # Model = should match the deployment name you chose for your 0125-Preview model deployment
          response_format={ "type": "json_object" },
          messages=self.messages
        )
        insights = gpt_response.choices[0].message.content
        return str(insights)
    #与API交互，返回结果
    def get_gpt_response(self,client, deployment_name, messages):
        try:
            response = client.chat.completions.create(
                model=deployment_name,
                messages=messages,
                temperature=0.7,
                max_tokens=3000,  # Adjusted for potentially longer responses
                top_p=1
            )

            if response and response.choices and len(response.choices) > 0:
                return response.choices[0].message.content
            
            else:
                print("Error: No valid response")
                return None
            
        except Exception as e:
            print(f"Error during API call: {e}")
            return None

    #给输入增加prompt
    def get_prompt_for_comments(self,comments, prompt):
        messages = [{ "role": "system", "content": prompt}]
        if isinstance(comments, str):
            messages.append({"role": "user", "content": comments})
        elif isinstance(comments, list):
            for comment in comments:
                messages.append({"role": "user", "content": comment})
        else:
            raise ValueError("comments must be a list or a string.")
            
        return messages
# print("agent初始化")
# print("-------------------test")
agent=Agent()
gpt_res=agent.chat("Bing的dau是多少 ", generate_prompt())
print(gpt_res)
# res_json=json.loads(gpt_res)
# with open("test.json","w",encoding="utf-8") as f:
#     json.dump(res_json,f,ensure_ascii=False)
# print("-------------------")

{"Analysis": {"input": "Bing的dau是多少", "description": "获取昨日（若今日为2024年6月20日，则昨日为2024年6月19日），Bing-Android和Bing-IOS应用程序的日活跃用户数（DAU）", "SQL": "SELECT \n    metrics_date, \n    SUM( CAST( metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION ) ) AS dau \nFROM \n    sapphire.sapphire_engagement_metrics_master \nWHERE \n    metrics_date = '2024-06-19' AND \n    period = 1 AND \n    market = '#Overall#' AND \n    os_version = '#Overall#' AND \n    device_model = '#Overall#' AND \n    client_version = '#Overall#' AND \n    client_build_type = '#Overall#' AND \n    install_channel_l1 = '#Overall#' AND \n    install_channel_l2 = '#Overall#' AND \n    install_channel_l3 = '#Overall#' AND \n    install_channel_l4 = '#Overall#' AND \n    first_launch_source = '#Overall#' AND \n    launch_source = '#Overall#' AND \n    application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B') \nGROUP BY \n    metrics_date;", "date": ["2024-06-19", "2024-06-19"], "result_count": 1}}


In [88]:
def generate_miniapp_prompt():
    miniapp_top_10_dau_query="""
    select metrics_date::date AS "time", dau as "DAU", b.mini_app_name
        from (
            SELECT 
                metrics_date, mini_app, sum(dau) as dau
            FROM sapphire.sapphire_mini_apps_dau_master
            WHERE metrics_date = '2024-06-07'
                AND market = '#Overall#'
                AND first_launch_source = '#Overall#'
                AND client_version = '#Overall#'
                AND install_source = '#Overall#'
                AND os_version = '#Overall#'
                AND client_build_type = '#Overall#'
                AND application_package = '#Overall#'
                group by metrics_date, mini_app
        ) as a 
        inner join (
            SELECT mini_app_id, mini_app_name 
            FROM sapphire.sapphire_mini_apps 
            where is_in_appstarter='true' and is_in_prod = 'true' and date = '2024-06-07'
            GROUP BY mini_app_id, mini_app_name
          ) as b
        on lower(a.mini_app) = lower(b.mini_app_id)
    order by metrics_date, dau desc
    LIMIT 10;
    """
    json_fewshot=[
      {
        "Analysis": {
            "input":"dau排名前十的miniapp？",
            "description":"获取在2024年6月7日这一天，日活跃用户数（DAU）排名前十的miniapp",
            "SQL": miniapp_top_10_dau_query,
            "date": ["2024-06-07","2024-06-07"],
            "result_count": 10,
        },
      },
    ]


    prompt_dau = """
    Context: 
    Now we have the user data for some apps, these app are search apps, it intagrated the copilot for AI.
    User can also search for information, this app get revenue by ads, and it also have many miniapps for different feature.
    We did some AB test based on user interaction data from an app, 
    which includes various metrics such as user behavior, codex (AI) behavior, and search behavior. 

    The data for each user is structured as a sentence.

    The goal is to generate SQL and more information.
    The query should focus on：
    market:
    Format is "en-us",first two is language, last two is country(xl means other counrty that we can not get info.)
    Eg."en-us" means American people speak English.

    For `application_id`：
    - the user will describe an app name, and you need to use the corresponding ID when you search. The mappings are as follows:
       - 'Bing-Android' -> '2130688B018F4B44BBED68E7A42BBA1E'
       - 'Bing-IOS' -> 'AE427635ADC245AE973038BCB3D7C21B'
       - 'Start-Android' -> '4DC5714ABCAD449BA13A9B701A3CF296'
       - 'Start-IOS' -> '4A5B528B59954AAE8725B509A41FBF1A'
       - 'Copilot-Android' -> 'F185A93DE6764B098D55089F610A3FB8'
       - 'Copilot-IOS' -> 'FC320C411FC12CD4DFBE9A00F3161364'

       If the user does not specify the OS name, you need to get both IDs for the app. For example, if the user inputs "Bing", you should use ['2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B'].

    metrics_date： if not set in the content, it is today-1, format is "2024-06-07".
    For example, if I don't add anything in the input, you need to get today's date, if today is "2024-06-20",you can use metrics_date = "2024-06-19"
    - `metrics_date`: If not set in the content, it should be yesterday's date in the format "YYYY-MM-DD". For example, if today's date is "2024-06-20", then `metrics_date` should be "2024-06-19".
    - Always use the exact date in the SQL query, do not use relative date expressions like `current_date - INTERVAL '1 day'`.

    date: Add date query when you search in the sapphire.sapphire_mini_apps


    For `mini_app`：
    - mini_app in sapphire.sapphire_mini_apps_dau_master == mini_app_id in sapphire.sapphire_mini_apps.
    - Use subqueries to isolate data and avoid repeated rows.
    - First, aggregate data from `sapphire.sapphire_mini_apps_dau_master`.
    - Then, join the aggregated data with `sapphire.sapphire_mini_apps` on the `mini_app_id` field, ensuring to filter by the correct date `metrics_date` in the subquery..

    there are some miniapp example in the database:
        ('HomePageFeed',)
        ('Rewards',)
        ('Weather',)
        ('NC-Settings',)
        ('Scaffolding: Settings (NativeComponent)',)
        ('Wallpapers',)
        ('Designer (Image Creator)',)
        ('Image Creator',)
        ('Bing Image Creator',)
        ('Cash Back',)
        ('Shopping: Cash Back',)


    Data Definition:
    If you need to get all data in this type, use "#Overall#". eg. market="#overall".
    Otherwise, if you need to specify type, use your type. eg. market="en-us".Not only in "market", use it in all type.

    Here is the schema of table(sapphire.sapphire_mini_apps_dau_master):                                                                                                                                                                                                                                                                
        metrics_date         | 2022-01-01
        application_id       | AE427635ADC245AE973038BCB3D7C21B
        mini_app             | C58CEA7EF6E89CA39F9401EDB12D241D
        market               | #Overall#
        first_launch_source  | #Overall#
        client_version       | #Overall#
        install_source       | #Overall#
        os_version           | #Overall#
        client_build_type    | #Overall#
        dau                  | 20
        create_at            | 2022-07-19 11:24:15.454516+00:00
        updated_at           | 2022-07-19 11:24:15.454516+00:00
        application_package  | #Overall#
        
    Default value (remember to take in every query if no specific):
        AND market = '#Overall#'
        AND first_launch_source = '#Overall#'
        AND client_version = '#Overall#'
        AND install_source = '#Overall#'
        AND os_version = '#Overall#'
        AND client_build_type = '#Overall#'
        AND application_package = '#Overall#'
        
    Here is the schema of table(sapphire.sapphire_mini_apps):
        umid                 | 61
        mini_app_id          | C58CEA7EF6E89CA39F9401EDB12D241D
        mini_app_name        | Weather
        mini_app_category    | Productivity
        is_in_appstarter     | True
        is_in_prod           | True
        platform_list        | Android,iOS
        build_type_list      | daily,dev,dogfood,production
        date                 | 2021-09-16


    Output Format: 
    Each set of data returns one json, followed by an example based on the provided data.

    result_count: How many result will get after query, for example, if group by 2 date and 3 application, it will get 2*3=6 columns.
    if you guess the column will more than 100, eg. group by market, please use 100.

    Specifications:
    - Analyze the input and describe their demand.
    - Use these column to identify and describe user demand.
    - Provide a detailed and correct pgsql query for each input.
    - Limit the output length to avoid excessive verbosity.
    - Refer the schema of table. Include the default conditions in every query to ensure accurate calculation by considering all categories
    - To calculate user count, always use `sum(dau) as dau`
    - Do not use `#Overall#` for `application_id`. It should always be a specific ID or a list of IDs.
    
    Task Examples: 
    """

    for idx,shot in enumerate(json_fewshot):
        prompt_dau+="Input"+str(idx)+"\n"+shot["Analysis"]["input"]+":\n"
        prompt_dau+="Output"+str(idx)+"\n"+json.dumps(shot,ensure_ascii=False)+":\n"
        
    return prompt_dau

agent=Agent()
gpt_res=agent.chat("Bing APP 在6月25日的rewards的dau是多少 ", generate_miniapp_prompt())
print(gpt_res)

{"Analysis": {
    "input": "Bing APP 在6月25日的rewards的dau是多少",
    "description": "查询2024年6月25日，在Bing APP（Android和iOS平台）的'rewards' miniapp的日活跃用户数（DAU）",
    "SQL": "\n    SELECT \n        SUM(a.dau) AS \"DAU\", b.mini_app_name\n    FROM sapphire.sapphire_mini_apps_dau_master AS a\n    INNER JOIN (\n        SELECT mini_app_id, mini_app_name\n        FROM sapphire.sapphire_mini_apps \n        WHERE mini_app_name = 'Rewards' \n            AND date = '2024-06-25' \n        GROUP BY mini_app_id, mini_app_name \n    ) AS b ON lower(a.mini_app) = lower(b.mini_app_id)\n    WHERE a.metrics_date = '2024-06-25'\n        AND a.application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B')\n        AND market = '#Overall#'\n        AND first_launch_source = '#Overall#'\n        AND client_version = '#Overall#'\n        AND install_source = '#Overall#'\n        AND os_version = '#Overall#'\n        AND client_build_type = '#Overall#'\n        AND application_package = '#Ov

In [89]:
res_json=json.loads(gpt_res)
# with open("test.json","w",encoding="utf-8") as f:
#     json.dump(res_json,f,ensure_ascii=False)
# print("-------------------")
# 提取 SQL 部分
sql_query = res_json['Analysis']['SQL']

print("----Extracted SQL Query:")
print(sql_query)

----Extracted SQL Query:

    SELECT 
        SUM(a.dau) AS "DAU", b.mini_app_name
    FROM sapphire.sapphire_mini_apps_dau_master AS a
    INNER JOIN (
        SELECT mini_app_id, mini_app_name
        FROM sapphire.sapphire_mini_apps 
        WHERE mini_app_name = 'Rewards' 
            AND date = '2024-06-25' 
        GROUP BY mini_app_id, mini_app_name 
    ) AS b ON lower(a.mini_app) = lower(b.mini_app_id)
    WHERE a.metrics_date = '2024-06-25'
        AND a.application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B')
        AND market = '#Overall#'
        AND first_launch_source = '#Overall#'
        AND client_version = '#Overall#'
        AND install_source = '#Overall#'
        AND os_version = '#Overall#'
        AND client_build_type = '#Overall#'
        AND application_package = '#Overall#'
    GROUP BY b.mini_app_name;



In [90]:
agentSQL.chat(sql_query,debug=True)

PGSQL Connection successful

    SELECT 
        SUM(a.dau) AS "DAU", b.mini_app_name
    FROM sapphire.sapphire_mini_apps_dau_master AS a
    INNER JOIN (
        SELECT mini_app_id, mini_app_name
        FROM sapphire.sapphire_mini_apps 
        WHERE mini_app_name = 'Rewards' 
            AND date = '2024-06-25' 
        GROUP BY mini_app_id, mini_app_name 
    ) AS b ON lower(a.mini_app) = lower(b.mini_app_id)
    WHERE a.metrics_date = '2024-06-25'
        AND a.application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B')
        AND market = '#Overall#'
        AND first_launch_source = '#Overall#'
        AND client_version = '#Overall#'
        AND install_source = '#Overall#'
        AND os_version = '#Overall#'
        AND client_build_type = '#Overall#'
        AND application_package = '#Overall#'
    GROUP BY b.mini_app_name;


    SELECT 
        SUM(a.dau) AS "DAU", b.mini_app_name
    FROM sapphire.sapphire_mini_apps_dau_master AS a
    INN

[(Decimal('501036'), 'Rewards')]

In [2]:

import psycopg2
from psycopg2 import sql
import json
# 全局数据库连接参数
DB_CONFIG = {
    'host': 'bingviz-metrics-storage-dup.postgres.database.azure.com',
    'port': '5432',
    'dbname': 'metrics',
    'user': 'bingviz_metrics',
    'password': "xb;Rt&F4~]XWj:@r-7.df'QD"
}
class AgentPGSQL():
    def __init__(self):
        print('init_agentSQL')
        self.schema_name = 'sapphire'
        self.table_name = 'sapphire_engagement_metrics_master'
        self.conn = self.connect_to_postgres()
        self.cursor = self.conn.cursor()
        # 关闭游标和连接
    def __del__(self) -> None:
        print('del-运行')
        self.cursor.close()
        self.conn.close()
    def chat(self,sql_query:str,debug=False)-> str:
        self.conn = self.connect_to_postgres()
        self.cursor = self.conn.cursor()
        """
        Decode the output of GPT
        """
        err_tb_names=['your_table_name',r"{}.{}","table_name"]
        for i  in err_tb_names:
            if i in sql_query:
                sql_query=sql_query.replace(i,"your_schema.your_table")
        sql_query=sql_query.replace("your_schema.your_table", f"{self.schema_name}.{self.table_name}")
        
        if debug==False:
            sql_query,value_dict=self.str2json(sql_query)
        if sql_query is None:
            return "GPT 生成结果JSON 解码错误"
        else:
            print(sql_query)
        with open("query.json","w",encoding="utf-8") as f:
            json.dump({"query":sql_query},f,ensure_ascii=False)
        
        # 执行 SQL 查询
        ana_res=self.fetch_complex_data(self.cursor, sql_query=sql_query)
        # 提交事务
        if ana_res is None:
            return "GPT 生成的代码无法运行，哈哈哈。"
        return ana_res

    def connect_to_postgres(self):
        """
        Connects to the PostgreSQL database using global DB_CONFIG and returns the connection and cursor.
        """
        try:
            # 连接到 PostgreSQL 数据库
            conn = psycopg2.connect(**DB_CONFIG)
            print("PGSQL Connection successful")
            return conn
        except Exception as e:
            print(f"Error: {e}")
            return None
    def str2json(self,input_str:str,target='DAU_Analysis'):
        # exa_j['DAU_Analysis'][0]['SQL']['query']
        value_dict={}
        try:
            print(type(input_str))
            ana_res=json.loads(input_str)
            sql_query=ana_res[target][0]["SQL"]['query']
            # exa_j['DAU_Analysis'][0]['SQL']['query']
            if "date" in ana_res:
                value_dict["date"]=ana_res["date"]
            return sql_query,value_dict
        except Exception as e:
            print(f"Error: {e}")
            print("解码错误有问题！")
            return None,None
        
    
    def execute_query(self,cursor, query):
        """
        Executes a given SQL query using the provided cursor and returns the results.
        """
        try:
            cursor.execute(query)
            return cursor.fetchall()
        except Exception as e:
            print(f"Error: {e}")
            return None
    def fetch_complex_data(self, cursor, sql_query):
        """
        Fetches complex data based on the provided criteria.
        """
        query = sql.SQL(sql_query)
        print(sql_query)
        results = self.execute_query(cursor, query)
        if results:
            for row in results:
                print(row)
            return results
        else:
            print(f"No results found for the query.")
            return None

# agentPGSQL=AgentPGSQL()
# example=r"""{"DAU_Analysis": [{"SQL": {"query": "SELECT \n    metrics_date,\n    application_id,\n    SUM(CAST(metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION)) AS \"dau\"\n FROM \n    {}.{}\nWHERE metrics_date BETWEEN '2024-06-09' AND '2024-06-09'\n    AND period = 1\n    AND os_version = '#Overall#'\n    AND device_model = '#Overall#'\n    AND client_version = '#Overall#'\n    AND client_build_type = '#Overall#'\n    AND install_channel_l1 = '#Overall#'\n    AND install_channel_l2 = '#Overall#'\n    AND install_channel_l3 = '#Overall#'\n    AND install_channel_l4 = '#Overall#'\n    AND mini_app = ''\n    AND first_launch_source = '#Overall#'\n    AND launch_source = '#Overall#'\n    AND market = '#Overall#'\n    AND application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B')\nGROUP BY metrics_date, application_id"}}, "additional_metrics"], "DAU": "dau", "startdate": "2024-06-04", "enddate": "2024-06-04", "application_id_list": ["2130688B018F4B44BBED68E7A42BBA1E", "AE427635ADC245AE973038BCB3D7C21B", "4DC5714ABCAD449BA13A9B701A3CF296", "4A5B528B59954AAE8725B509A41FBF1A"], "query_conclusion": "查询指定的四个应用程序（Bing-Android、Bing-IOS、Start-Android、Start-IOS）在2024年6月4日的日活跃用户数（DAU）。", "result_count": "4", "market_analysis": {"enabled": false}}"""
# print(example[44:46])
# example=example.replace(r"{}.{}","tablename")
# print(agentPGSQL.chat(example))

In [3]:
agentSQL=AgentPGSQL()

init_agentSQL
PGSQL Connection successful


In [4]:
market_top_10_query="""
SELECT market,
SUM( CAST( metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION ) ) AS dau 
FROM 
your_schema.your_table 
WHERE metrics_date BETWEEN  '2024-06-09' AND '2024-06-09'
AND application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B')
GROUP BY market
ORDER BY dau DESC
LIMIT 10;"""

In [5]:
agentSQL.chat(market_top_10_query,debug=True)

PGSQL Connection successful

SELECT market,
SUM( CAST( metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION ) ) AS dau 
FROM 
sapphire.sapphire_engagement_metrics_master 
WHERE metrics_date BETWEEN  '2024-06-09' AND '2024-06-09'
AND application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B')
GROUP BY market
ORDER BY dau DESC
LIMIT 10;

SELECT market,
SUM( CAST( metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION ) ) AS dau 
FROM 
sapphire.sapphire_engagement_metrics_master 
WHERE metrics_date BETWEEN  '2024-06-09' AND '2024-06-09'
AND application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B')
GROUP BY market
ORDER BY dau DESC
LIMIT 10;
('#Overall#', 705696946.0)
('en-xl', 35851980.0)
('en-us', 27332840.0)
('es-xl', 24975081.0)
('en-in', 22389759.0)
('pt-br', 15275964.0)
('zh-cn', 11140802.0)
('es-mx', 8449721.0)
('ja-jp', 8026148.0)
('de-de', 7703039.0)


[('#Overall#', 705696946.0),
 ('en-xl', 35851980.0),
 ('en-us', 27332840.0),
 ('es-xl', 24975081.0),
 ('en-in', 22389759.0),
 ('pt-br', 15275964.0),
 ('zh-cn', 11140802.0),
 ('es-mx', 8449721.0),
 ('ja-jp', 8026148.0),
 ('de-de', 7703039.0)]